# Models with three faults and two layers
This example tries to visualize how simply _GeoMeshPy_ can export results from _Gempy_. <br>
This notebook include a complicated model with two layers and three fault. <br>
To see how _Gempy_ works, please see https://github.com/cgre-aachen/gempy

In [ ]:
import matplotlib.pyplot as plt
import copy
import math
import gempy as gp
import numpy as np
from numpy import savetxt
geo_model = gp.create_model('3_F')
gp.init_data(geo_model, [0., 1400., 0., 1000., -1800., -1000.], [40, 40, 40],
            path_i = 'thr_F_interfaces.csv',
            path_o = 'thr_F_orientations.csv');
geo_model.add_surfaces('Reservoir')
gp.map_series_to_surfaces(geo_model,
                         {"Fault1_series":'fault1',
                          "Fault2_series":'fault2',
                          "Fault3_series":'fault3',
                          "Strati_series":('Cap_rock', 'Reservoir')})
geo_model.set_is_fault(['Fault1_series', 'Fault2_series', 'Fault3_series'], change_color=False)
fr = np.array([[False, False, False, True, True],
               [False, False, False, True, True],
               [False, False, False, False, False],
               [False, False, False, False, False],
               [False, False, False, False, False]])
geo_model.set_fault_relation(fr)
gp.set_interpolator(geo_model,
                         compile_theano=True,
                         theano_optimizer='fast_compile',
                         verbose=[])
sol = gp.compute_model(geo_model)
extent = geo_model.grid.regular_grid.extent
resolution = geo_model.grid.regular_grid.resolution.reshape(-1,1)
df=geo_model.series.df
if len (np.unique (sol.fault_block))>1:
    no_of_faults=df.groupby(by='BottomRelation').count().iloc[1,0]
else:
    no_of_faults=0

surfaces=geo_model.surface_points.df['surface']
if no_of_faults==0:
    surfaces_layer=[i for i in surfaces.unique()]
else:
    surfaces_layer=[i for i in surfaces.unique()[no_of_faults:]]
    fault_name=[i for i in surfaces.unique()[:no_of_faults]]
grid=geo_model.grid.values
z_resolution = abs (grid[0,-1] - grid[1,-1])
res_x=abs(extent[1]-extent[0])/resolution[0,0]
surfaces_layer.append('Basement')
lith_blocks = np.array([])
ver=[]
fault_ind=[]
n_iter = 10
for i in range(n_iter):
#     Initialization of the Gempy model
    df_int_X      = copy.copy(geo_model.surface_points.df['X'])
    df_int_Y      = copy.copy(geo_model.surface_points.df['Y'])
    df_int_Z      = copy.copy(geo_model.surface_points.df['Z'])
    df_or_X       = copy.copy(geo_model.orientations.df['X'])
    df_or_Y       = copy.copy(geo_model.orientations.df['Y'])
    df_or_Z       = copy.copy(geo_model.orientations.df['Z'])
    df_or_dip     = copy.copy(geo_model.orientations.df['dip'])
    df_or_azimuth = copy.copy(geo_model.orientations.df['azimuth'])
    surfindexes = list(geo_model.surface_points.df.index)
    orindexes = list(geo_model.orientations.df.index)
    geo_model.modify_surface_points(surfindexes, X=df_int_X, Y=df_int_Y, Z=df_int_Z)
    geo_model.modify_orientations(orindexes, X=df_or_X, Y=df_or_Y, Z=df_or_Z,dip = df_or_dip, azimuth = df_or_azimuth)
  
    fault_3_surfpoints = geo_model.surface_points.df.surface.isin(['fault3'])
    indexes_Fa_3_sp = geo_model.surface_points.df[fault_3_surfpoints].index
    fault_3_orient = geo_model.orientations.df.surface.isin(['fault3'])
    index_Fa_3_o = geo_model.orientations.df[fault_3_orient].index
#     Randomization_Method
    if i == 0: # in the first step we do not want any change
        std1=std2=0
    else:
        std1=10
        std1=5
    rand1 = np.random.uniform(-std1, std1, size=1)
    rand2 = np.random.uniform(-std2, std2, size=1)
#     Randomized_input    
    a = geo_model.surface_points.df['Z'].values[fault_3_surfpoints][0] + rand1
    b = geo_model.surface_points.df['Z'].values[fault_3_surfpoints][1] + rand1
    new_Z_fa_3 = np.array([a,b])
    new_Z_fa_3 = new_Z_fa_3.flatten()
    new_Y_fa_3 = geo_model.surface_points.df['Y'].values[fault_3_surfpoints]
    new_X_fa_3 = geo_model.surface_points.df['X'].values[fault_3_surfpoints]
    new_o_fa_3 =geo_model.orientations.df['azimuth'].values[fault_3_orient] + rand2
#     Modifier
    geo_model.modify_surface_points(indexes_Fa_3_sp, Z = new_Z_fa_3)
    geo_model.modify_orientations(index_Fa_3_o, azimuth = new_o_fa_3)
#     this block updates the model
    geo_model.set_is_fault(['Fault1_series', 'Fault2_series', 'Fault3_series'], change_color=False)
    fr = np.array([[False, False, False, True, True],
                   [False, False, False, True, True],
                   [False, False, False, False, False],
                   [False, False, False, False, False],
                   [False, False, False, False, False]])
    geo_model.set_fault_relation(fr)
    geo_model.update_to_interpolator()
    sol=gp.compute_model(geo_model)
    # Export Block
    ver.append(geo_model.solutions.vertices)
    lith_blocks = np.append(lith_blocks, geo_model.solutions.lith_block)
    fault_ind.append (np.hstack([grid,np.round(sol.fault_block.T[0:sol.grid.values.shape[0]])]))

In [ ]:
lith_blocks = lith_blocks.reshape(n_iter, -1)
lays_fault_name=geo_model.surface_points.df.loc[:, 'surface'].unique()
all_vers=[list(column) for column in zip(*ver)]
df=geo_model.series.df
no_of_faults=df.groupby(by='BottomRelation').count().iloc[1,0]
name_of_faults=lays_fault_name[0:no_of_faults].tolist()
name_of_layers=lays_fault_name[no_of_faults:].tolist()

In [ ]:
from GeoMeshPy import vmod # the class vmod allows you for doing all the required calculations

In [ ]:
z_resolution = 20
fr = np.array([[True],
               [True],
               [False]])
name_of_layers = np.array(['Cap_rock', 'Reservoir'])
model_faulted = vmod.vertice_modifier(n_iter, no_of_faults, all_vers, name_of_layers, z_resolution, fr, extent, resolution)
sub_fourc_list = model_faulted.faults_corners()[0]
len_fal = model_faulted.faults_corners()[1]
new_result_list = model_faulted.contact_generator()[0]
length_layers = model_faulted.contact_generator()[1]
repre_pts = model_faulted.contact_generator()[2]

The visialization in the next block reveals some facts: <br>
1. There are extra redundant point in verticed coming out of Gempy
2. Gempy does not cut layers when while they are relocated by fault. in the other words,
layers just strech along the fault surfaces.
3. Almost caused by 2, contact of the layer is uniform while there ae two faults cutting it. To solve
this issue, surfaces are deivided based on the existing fault.
For example, in this case the cotact should be split into three patches which are 
shown by different color in the visualization cell. If you zoom in the contact of 
layer and two cutting fault, you will see some vertices of Gempy there.

In [5]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
%matplotlib qt5

fig = plt.figure()
ax = fig.add_subplot (111, projection="3d")

# Gempy utputs for the only layers and third fault
Gempy_out_layer = all_vers[3][0]
Gempy_out_fault_3 = all_vers[2][0]
x2 = Gempy_out_layer[:,0]; y2 = Gempy_out_layer[:,1]; z2 = Gempy_out_layer[:,2]
ax.scatter3D(x2,y2,z2, color='k', s=1, label='Raw output')
x2 = Gempy_out_fault_3[:,0]; y2 = Gempy_out_fault_3[:,1]; z2 = Gempy_out_fault_3[:,2]
ax.scatter3D(x2,y2,z2, color='k', s=1, label='Raw output')


# cleaned and separated data coming from GeoMeshPy
faults = np.array(sub_fourc_list[0])
f1 = faults[:4,:]
f2 = faults[4:8,:]
f3 = faults[8:12,:]
x2=faults[:,0]; y2=faults[:,1]; z2=faults[:,2]
ax.scatter3D(x2,y2,z2, color='r', s=10, marker= '*', label='Faults corners')
ax.plot_surface(np.array([[f1[0,0], f1[1,0]],  [f1[3,0], f1[2,0]]]),
                np.array([[f1[0,1], f1[1,1]],  [f1[3,1], f1[2,1]]]),
                np.array([[f1[0,2], f1[1,2]],  [f1[3,2], f1[2,2]]]), color='b', alpha = 0.5, label='Injection Fault')
ax.plot_surface(np.array([[f2[0,0], f2[1,0]],  [f2[3,0], f2[2,0]]]),
                np.array([[f2[0,1], f2[1,1]],  [f2[3,1], f2[2,1]]]),
                np.array([[f2[0,2], f2[1,2]],  [f2[3,2], f2[2,2]]]), color='g', alpha = 0.5, label='Production Fault')
ax.plot_surface(np.array([[f3[0,0], f3[1,0]],  [f3[3,0], f3[2,0]]]),
                np.array([[f3[0,1], f3[1,1]],  [f3[3,1], f3[2,1]]]),
                np.array([[f3[0,2], f3[1,2]],  [f3[3,2], f3[2,2]]]), color='r', alpha = 0.8, label='Connecting Fault')

po = np.array(new_result_list[0])
le = np.array(length_layers[0]).astype('int')
le = np.cumsum (le)
ax.scatter3D(po[:le[0],0],po[:le[0],1],po[:le[0],2], facecolor='None', color='orange', s=5, linewidths=0.5)
ax.scatter3D(po[le[0]:le[1],0],po[le[0]:le[1],1],po[le[0]:le[1],2],facecolor='None',linewidths=0.5,  color='b', s=5)
ax.scatter3D(po[le[1]:le[2],0],po[le[1]:le[2],1],po[le[1]:le[2],2],facecolor='None',linewidths=0.5, color='r', s=5)


# representative point proposed by GeoMeshPy
reps = np.array(repre_pts[0])[:,:-1].astype('float')
ax.scatter3D(reps[:3,0],reps[:3,1],reps[:3,2], marker= '*', color='orange', s=50)
ax.scatter3D(reps[3:,0],reps[3:,1],reps[3:,2], marker= '*', color='c', s=50)


ax.set_yticks([0, 500, 1000])
ax.set_xticks([0, 700, 1400])
ax.set_zticks([-1000, -1400, -1800])
ax.set_ylim(0, 1000)
ax.set_xlim([0, 1400])
ax.set_zlim([-1800, -1000])
ax.tick_params(axis='both', which='major', labelsize=10)
ax._facecolors2d = ax._facecolor
ax.grid(None)
plt.show()
ax.view_init(5, 270)

In [ ]:
# in this block you can export outputs of GeoMeshPy to avoir running GemPY and GeomeshPy again
from numpy import savetxt
sets = zip(sub_fourc_list, new_result_list, repre_pts, len_fal)
for ind, (crn_fal, vertices, rep_pnt, len_fals) in enumerate(sets):
    savetxt(f'fal_crn_{ind}.csv', np.array(crn_fal), delimiter=',')
    savetxt(f'vertices_{ind}.csv', np.array(vertices), delimiter=',')
    savetxt(f'rep_pnt_{ind}.csv', np.array(rep_pnt), delimiter=',', fmt="%s")
    savetxt(f'len_fals_{ind}.csv', np.array(len_fals), delimiter=',')
savetxt('len_layer.csv', length_layers, delimiter=',')

In [4]:
import numpy as np
import copy
extent = np.array([0., 1400., 0., 1000., -1800., -1000.])
from GeoMeshPy import vmod
n_iter=10

# this model has two wells peneterating into the two faults

wells=[[600.,500.,-1350.], [550.,500.,-1450.],\
       [850.,500.,-1350.], [900.,500.,-1450.]]
wel_iter=np.split (np.tile(wells, (n_iter, 1)), n_iter)
wells_cord=[i.tolist() for i in wel_iter]
well_points=[[2., 2]]
well_p_iter=np.split (np.tile(well_points, (n_iter, 1)), n_iter)
well_points=[i.tolist() for i in well_p_iter]
wl_names=['W_1', 'W_2']


name_of_faults=['fault1', 'fault2', 'fault3']
no_of_faults= len (name_of_faults)
sub_fourc_list=[]
new_result_list=[]
repre_pts=[]
len_fal=[]
from numpy import genfromtxt
length_layers=genfromtxt('len_layer.csv', delimiter=',').tolist()
import glob
files_fal_cr = glob.glob("fal_crn_*.csv")
files_fal_crn= sorted(files_fal_cr, key=lambda name: int(name[8:-4]))
files_ve = glob.glob("vertices_*.csv")
files_ver = sorted(files_ve, key=lambda name: int(name[9:-4]))
files_repr= glob.glob("rep_pnt_*.csv")
files_repre= sorted(files_repr, key=lambda name: int(name[8:-4]))
files_le= glob.glob("len_fals_*.csv")
files_len= sorted(files_le, key=lambda name: int(name[9:-4]))
set_names = zip(files_fal_crn, files_ver, files_repre, files_len)
for name_fal, name_ver, name_rep, name_len in set_names:
    fal_crn=np.around(genfromtxt(name_fal, delimiter=','), decimals=6)
    sub_fourc_list.append(fal_crn.tolist())
    new_result_list.append(np.around(genfromtxt(name_ver, delimiter=','), decimals=6).tolist())
    repre_pts.append(genfromtxt(name_rep, delimiter=',', dtype=str).tolist())
    len_fal.append([genfromtxt(name_len, delimiter=',').tolist()])

This cell will add another representative point to the model becasue you have <br>
a fault which is just passing through the model. Thins fault in GMSH will make an extra volume  <br>
but the volume is not close to any surface.

In [ ]:
extra = ['750.', '500.', '-1700.', 'Reservoir']
for i in range (len(repre_pts)):
    repre_pts[i].insert(len(repre_pts[i]),extra)

In [ ]:
from numpy import savetxt
import gmsh
import itertools
from itertools import chain
gmsh.initialize()
if no_of_faults>0:
    def cleanup_and_mesh(entities_to_preserve):
        # remove all embedded constraints, i.e. the entities that are not on the
        # boundary of others
        entities = gmsh.model.getEntities()
        for e in entities:
            emb = gmsh.model.mesh.getEmbedded(e[0], e[1])
            gmsh.model.mesh.removeEmbedded([e])
            for p in entities_to_preserve:
                if p in emb:
                    gmsh.model.mesh.embed(p[0], [p[1]], e[0], e[1])
        # remove all surfaces, curves and points that are not connected to any
        # higher-dimensional entities
        gmsh.model.removeEntities(gmsh.model.getEntities(2), True)
        cc = gmsh.model.getEntities(1)
        for c in curves_to_preserve:
            cc.remove(c)
        gmsh.model.removeEntities(cc, True)
        gmsh.model.removeEntities(gmsh.model.getEntities(0))
        # get all surfaces that are not of type "Plane", i.e. all surfaces except the
        # box
        surfaces = [s[1] for s in gmsh.model.getEntities(2) if gmsh.model.getType(s[0], s[1])
                    != 'Plane']
        # also refine close to the wells
        surface_after = gmsh.model.getEntities(2)
        points=copy.deepcopy(surface_new_tag)
        check_values=[row[-1] for row in surface_after]
        extracted = []
        for sublist in points:
            second_vals = [sec for fir, sec in sublist]
            if all(val in check_values for val in second_vals):
                extracted.append(second_vals)
        fl=[item for sublist in extracted[6:] for item in sublist]
#         fl_sur.append(fl)
        contact_surfaces = list(set(surfaces) - set(fl))
        # create a distance + threshold mesh size field w.r.t. these surfaces
        gmsh.model.mesh.field.add("Distance", 1)
        gmsh.model.mesh.field.setNumbers(1, "SurfacesList", [sp_fls[0][0], sp_fls[1][1], sp_fls[2][1]])
        gmsh.model.mesh.field.setNumber(1, "Sampling", 100)
        gmsh.model.mesh.field.add("Threshold", 2)
        gmsh.model.mesh.field.setNumber(2, "InField", 1)
        gmsh.model.mesh.field.setNumber(2, "SizeMin", 30)
        gmsh.model.mesh.field.setNumber(2, "SizeMax", 100)
        gmsh.model.mesh.field.setNumber(2, "DistMin", 35)
        gmsh.model.mesh.field.setNumber(2, "DistMax", 100)
        gmsh.model.mesh.field.add("Distance", 3)
        gmsh.model.mesh.field.setNumbers(3, "PointsList", fal_wl_conection)
        gmsh.model.mesh.field.setNumber(3, "Sampling", 100)
        gmsh.model.mesh.field.add("Threshold", 4)
        gmsh.model.mesh.field.setNumber(4, "InField", 3)
        gmsh.model.mesh.field.setNumber(4, "SizeMin", 10.0)
        gmsh.model.mesh.field.setNumber(4, "SizeMax", 100)
        gmsh.model.mesh.field.setNumber(4, "DistMin", 10.)
        gmsh.model.mesh.field.setNumber(4, "DistMax", 50.)
        gmsh.model.mesh.field.add("Distance", 5)
        gmsh.model.mesh.field.setNumbers(5, "CurvesList", np.array(curves_to_preserve)[:,1].tolist())
        gmsh.model.mesh.field.setNumber(5, "Sampling", 1000)
        gmsh.model.mesh.field.add("Threshold", 6)
        gmsh.model.mesh.field.setNumber(6, "InField", 5)
        gmsh.model.mesh.field.setNumber(6, "SizeMin", 5.)
        gmsh.model.mesh.field.setNumber(6, "SizeMax", 100)
        gmsh.model.mesh.field.setNumber(6, "DistMin", 6)
        gmsh.model.mesh.field.setNumber(6, "DistMax", 100)
        gmsh.model.mesh.field.add("Distance", 7)
        gmsh.model.mesh.field.setNumbers(7, "SurfacesList", contact_surfaces)
        gmsh.model.mesh.field.setNumber(7, "Sampling", 100)
        gmsh.model.mesh.field.add("Threshold", 8)
        gmsh.model.mesh.field.setNumber(8, "InField", 7)
        gmsh.model.mesh.field.setNumber(8, "SizeMin", 15)
        gmsh.model.mesh.field.setNumber(8, "SizeMax", 100)
        gmsh.model.mesh.field.setNumber(8, "DistMin", 20)
        gmsh.model.mesh.field.setNumber(8, "DistMax", 100)
        gmsh.model.mesh.field.add("Min", 9)
        gmsh.model.mesh.field.setNumbers(9, "FieldsList", [2,4,6,8])
        gmsh.model.mesh.field.setAsBackgroundMesh(9)
        gmsh.option.setNumber("Mesh.MeshSizeMax", 100)
        # don't extend mesh sizes from boundaries and use new 3D algo
        gmsh.option.setNumber("Mesh.MeshSizeExtendFromBoundary", 0)
        gmsh.option.setNumber("Mesh.Algorithm3D", 10)
        gmsh.model.mesh.generate(3)
        
        # using representative points to create physical volumes
        rep=[list(x) for _,x in itertools.groupby(rep_pnt,lambda x:x[3])]
        vol_num=np.arange(1,1+len(rep))
        for ind, surfaces in enumerate (rep):
            tags=[]
            for sects in surfaces:
                eleTag = gmsh.model.mesh.getElementByCoordinates(float (sects[0]), float (sects[1]), float (sects[2]))[0]
                eleType, eleNodes, entDim, entTag = gmsh.model.mesh.getElement(eleTag)
                tags.append(entTag)
            gmsh.model.addPhysicalGroup(3, tags, vol_num[ind])
            gmsh.model.setPhysicalName(3, vol_num[ind], sects[-1])             
        for tag_nu, name in zip (sp_fls, name_of_faults):
            ps1 = gmsh.model.addPhysicalGroup(2, tag_nu)
            gmsh.model.setPhysicalName(2, ps1, name)   
        #adding wells as physical lines
        for lines, well_name in zip (sp_well, wl_names):
            l1 = gmsh.model.addPhysicalGroup(1, lines.tolist())
            gmsh.model.setPhysicalName(1, l1, well_name)            
        around_box=['in', 'out', 'front', 'back', 'bottom', 'top']       
        for tag_nu, name in zip (extracted[:6], around_box):
            ps1 = gmsh.model.addPhysicalGroup(2, tag_nu)
            gmsh.model.setPhysicalName(2, ps1, name)
        inj_prod=['Injection_point', 'Production_point']
        inj_prod_t=[points_to_preserve[0][1], points_to_preserve [len (sp_well[0])*2][1]]
        for t, po_n in zip (inj_prod_t, inj_prod):
            p1=gmsh.model.addPhysicalGroup(0, [t])
            gmsh.model.setPhysicalName(0, p1, po_n)
            
        connections=['Conj_1', 'Conj_2']
        for t, po_n in zip (fal_wl_conection, connections):
            p1=gmsh.model.addPhysicalGroup(0, [t])
            gmsh.model.setPhysicalName(0, p1, po_n)
            
            
        gmsh.write("Moded_3Faults_" + str(kk) + ".msh")
        gmsh.fltk.run()
    gmsh.initialize()
    degree = 3
    numPointsOnCurves = 10
    numIter = 10
    anisotropic = False
    tol2d = 0.00001
    tol3d = .1
    tolAng = 1
    tolCurv = 1
    maxDegree = 3
    maxSegments = 100
    sets = zip(sub_fourc_list, new_result_list, repre_pts, wells_cord, well_points, len_fal)
    for kk, (crn_fal, vertices, rep_pnt, well_cord, well_p, len_fals) in enumerate(sets):
        ar=np.array(vertices)
        l_tags=[]
        sp_fal_num=np.cumsum(np.array(len_fals).astype('int'))
        sp_fal=np.split (np.array(crn_fal), sp_fal_num[:-1])
        gmsh.model.occ.addBox(min(ar[:,0]),crn_fal[0][1],crn_fal[0][2],max(ar[:,0])-min(ar[:,0]),
                              crn_fal[1][1]-crn_fal[0][1],crn_fal[2][2]-crn_fal[0][2])

        for i in range (len(sp_fal)):
            for [x, y, z] in sp_fal[i]:    
                gmsh.model.occ.addPoint(x, y, z)
        tag_p_fal=np.arange(9, len (crn_fal)+9)
        tag_sp_fal=np.split (tag_p_fal, sp_fal_num[:-1])
        for i in tag_sp_fal:
            for j in range (len(i)):
                if j==len(i)-1:
                    gmsh.model.occ.addLine (i[j], i[0])
                else:
                    gmsh.model.occ.addLine (i[j], i[j+1])
        tag_l_fal=np.arange(13, len (crn_fal)+13)
        tag_sl_fal=np.split (tag_l_fal, sp_fal_num[:-1])
        for i in tag_sl_fal:
            lop=i.tolist()
            gmsh.model.occ.addCurveLoop(lop, lop[0]*10)
            gmsh.model.occ.addSurfaceFilling(lop[0]*10, lop[0]*10)

        spl_num=np.cumsum(length_layers[kk]).tolist()[:-1] # each cloud of points is separated
        spl_num=[int (i) for i in spl_num]
        sep_ar=np.split(ar,spl_num)
        for ind, point_clouds in enumerate (sep_ar):
            i_l=point_clouds.tolist()
            for [x, y, z] in i_l:
                gmsh.model.occ.addPoint(x, y, z)
            if len (point_clouds)>3:
                y_sub=np.unique(point_clouds[:,1].round(5),return_counts=True)[1]
                x_sub=np.unique(point_clouds[:,0].round(5),return_counts=True)[1]
                pts=[]
                for j in np.split (point_clouds, np.cumsum(x_sub)[:-1]):
                    if (j[0]!=j[-1]).any():
                        pts.append([j[0], j[-1]])
                for m in np.split (point_clouds[np.lexsort((point_clouds[:,0],point_clouds[:,1]))], np.cumsum(y_sub)[:-1]):
                    if (m[0]!=m[-1]).any():
                        pts.append([m[0], m[-1]])
                a=[[j.tolist() for j in i] for i in pts]
                b = list(chain.from_iterable(a))
                c=list(set(tuple(x) for x in b))
                d=[list(i) for i in c]
                f= [sublist for sublist in d]
                g=np.array(f)
                h=g[np.lexsort((g[:,1],g[:,0]))] # it include all the extrerior points of the cloud
                pnt=h[:,0:-1].tolist()
                arround_pts=vmod.vertice_modifier.rotational_sort(pnt, (np.mean(np.array(pnt)[:,0]),np.mean(np.array(pnt)[:,1])),True)
                tags=np.where((point_clouds[:,:-1]==np.array(arround_pts)[:,None]).all(-1))[1] + 1
                l_tags.append(len(tags))
                start_point=int (8+len(crn_fal)+np.sum(length_layers[kk][0:ind]))
                start_line=int (12+len(crn_fal)+1+np.sum(l_tags[0:ind]))
                for i in range (len(tags)): # this for loop creates the exterior lines of each cloud
                    if i!=len(tags)-1:
                        gmsh.model.occ.addSpline([tags[i]+start_point,tags[i+1]+start_point])
                    else:
                        gmsh.model.occ.addSpline([tags[i]+start_point,tags[0]+start_point])
                gmsh.model.occ.addCurveLoop([i for i in range (start_line, start_line+len(tags))], start_line*10)
                gmsh.model.occ.addSurfaceFilling(start_line*10, start_line*10,
                                                 [m for m in range (start_point+1, start_point+np.max(tags))
                                                  if m not in tags+start_point],
                                                 degree,
                                                 numPointsOnCurves,
                                                 numIter,
                                                 anisotropic,
                                                 tol2d,
                                                 tol3d,
                                                 tolAng,
                                                 tolCurv,
                                                 maxDegree,
                                                 maxSegments) # create surface by connecting exterior lines

                                                                                # and inclding interior ones
        gmsh.model.occ.synchronize()
        gmsh.option.setNumber('Geometry.ToleranceBoolean', 0.01)

        in_surf = gmsh.model.occ.getEntities(2)

        # TODO generalize here for all your wells:

        tag_well=np.arange(10000, 10000+len(np.array(well_cord)))
        well_p=np.array(well_p[0]).astype('int')
        tag_well_po=np.split (tag_well, np.cumsum(well_p)[:-1])
        well_po=np.split (np.array(well_cord), np.cumsum(well_p)[:-1])  
        for cord, tag_nu in zip (well_po, tag_well_po):
            for [x, y, z], num in zip (cord,tag_nu):
                gmsh.model.occ.addPoint(x, y, z, tag=num)
        well_l=well_p-1        
        tag_w_l=np.arange(5000, 5000+np.sum(well_l))
        wl=[i.tolist() for i in tag_w_l]
        tag_well_l=np.split (tag_w_l, np.cumsum(well_l)[:-1])

        for po, tag_num in zip (tag_well_po, tag_well_l):
            for i in range (len(po)-1):
                gmsh.model.occ.addLine (po[i], po[int(i+1)], tag=tag_num[i])
        in_wells=[(1, i) for i in tag_w_l]
        out_all=gmsh.model.occ.fragment(in_surf+in_wells, gmsh.model.occ.getEntities(3))#[1]
        out=out_all[1]
        surface_new_tag = out[0:len(in_surf)]
        c = out[len(in_surf):len(in_surf+in_wells)]
        curves_to_preserve = [item for sublist in c for item in sublist]
        gmsh.model.occ.synchronize()
        points_to_preserve=gmsh.model.getBoundary(curves_to_preserve, combined=False)
        line_sp=np.array([])
        for i in range (len(points_to_preserve)-1):
            if i%2!=0:
                if points_to_preserve[i][1]!=points_to_preserve[i+1][1]:
                    brk=int ((i+1)/2)
                    line_sp=np.append(line_sp, brk)
        sp_well=np.split(np.array(curves_to_preserve)[:,1],line_sp.astype('int'))
        preserving=[i[1] for i in points_to_preserve]
        pres_cord=[]
        for i in preserving:
            no_parametrization = []
            [x_p, y_p, z_p] = gmsh.model.getValue(0, i, no_parametrization)
            pres_cord.append([x_p, y_p, z_p])
        nons= np.where((np.array(pres_cord)==np.array(well_cord)[:,None]).all(-1))[1] # indices of new_points in wells
        # that are created by their connection with fault
        fal_wl_conection=list (set (np.delete(np.array(preserving), nons))) # tag of new_points in wells
        lst = [m for i,m in enumerate (pres_cord) if i not in nons.tolist()]
        p_ar = np.unique(np.array (lst), axis=0)

#         extracting fault indices
        fault_tag_num = [i[0]*10 for i in tag_sl_fal]
        ind_fault_surface = [x for x, y in enumerate(in_surf) if y[1] in fault_tag_num]
        sp_f = [i for ind, i in enumerate (surface_new_tag) if ind in ind_fault_surface]
        sp_fls = [[i[1] for i in j] for j in sp_f]
        cleanup_and_mesh(curves_to_preserve + points_to_preserve)
        gmsh.clear()
    gmsh.finalize()